In [137]:
import numpy as np

class ActivationFunctions:
        
    def ReLu(data):
        temp = np.array(data)
        return np.maximum(0,temp)
        
    def Sign(data):
        temp = np.array(data)
        return np.sign(temp)
            
    def Sigmoid(data):
        temp = np.array(data)
        return 1/(1+np.exp(-temp))
    def Default(data):
        temp = np.array(data)
        return temp
    def SoftMax(data):
        temp = np.exp(np.array(data))
        return temp/np.sum(temp)
        

class Layer:
    
    def __init__(self,shape,function=ActivationFunctions.Default) -> None:
        assert len(shape)==2
        assert isinstance(shape[0],int) and isinstance(shape[1],int)
        assert shape[0]>0 and shape[1]>0
        self.function=function
        self.shape = shape

        self._init_matrix()

    def __init__(self,shape_x:int,shape_y:int,function=ActivationFunctions.Default) -> None:
        assert shape_y>0 and shape_x>0

        self.shape = (shape_x,shape_y)
        self.function=function
        self._init_matrix()

    def _init_matrix(self):
        self.martix = np.random.randn(self.shape[0],self.shape[1])


    def forward(self,data,function=ActivationFunctions.Default):
        
        if function in [None,ActivationFunctions.Default]:
            function = self.function
        if function == None:
            return np.dot(data,self.martix)
        else:
            return function(np.dot(data,self.martix))
    

In [68]:
class Model:

    def __init__(self):
        self.layers=[]
        self.structure=[]
        self.learningFactor=0.02
    def addLayer(self,numberOfNeurons:int,function=ActivationFunctions.Default):
        assert numberOfNeurons>0
        # todo to validate function
        self.layers.append((numberOfNeurons,function))
        
    def compileModel(self):
        assert len(self.layers)>=2
        
        for i in range(len(self.layers)-1):
            self.structure.append(Layer(self.layers[i][0],self.layers[i+1][0],self.layers[i+1][1]))
    def forward(self,data):
        output=np.array(data)
        for i in self.structure:
            output = i.forward(output)
        return output

In [142]:
model =Model()
model.addLayer(5,ActivationFunctions.Sigmoid)
model.addLayer(5,ActivationFunctions.Sigmoid)
model.addLayer(1,ActivationFunctions.Sigmoid)

model.compileModel()
# for i in range(len(model.structure)):
#     print(model.structure[i].shape,end=f" {model.structure[i].function}\n ")
model.forward(np.array([1,2,3,4,5]))

<function ActivationFunctions.Sigmoid at 0x7fcd4caddd30>
<function ActivationFunctions.Sigmoid at 0x7fcd4caddd30>


array([0.38368753])

In [55]:
ActivationFunctions.Sign()

In [42]:
ActivationFunctions.SoftMax(np.linspace(-1,1,100))

1.0000000000000002


Taka jest docelowa funckja LOSS czyli taka którą minimalizujemy.

```1```- indidator function https://www.wikiwand.com/en/Indicator_function

```i``` iteruje sie po obserwacjach

```c``` iteruje sie po klasach

$p_{model} [y_i \in C_c ]$ prawdopodobieństwo, że obserwacja i należy do klasy c

$-1/N*\Sigma_{i=1} ^N \Sigma_{c=1} ^C 1_{y_{i} \in C_{c}} log(p_{model}[y_i \in C_{c}]) $